In [58]:
import numpy as np
import pandas as pd

from ast import literal_eval
import os, time

# Random Functions

In [54]:
RESULT_DIR = "final_results"
benchmark = "rational"

In [55]:
df = {}

for f_name in os.listdir(RESULT_DIR):
    try:
        benchmark_name = f_name.split("_")[1]
        tool_name =  f_name.split("_")[2]
        if benchmark_name == benchmark:
            df[tool_name] = pd.read_csv(os.path.join(RESULT_DIR, f_name)).sort_values('index')
    except Exception as e:
        pass

In [56]:
df_res = pd.DataFrame({
    "Function": df['faza']['formula'],
})
for tool in sorted(df):
    df_res[tool.capitalize()+"_output"] = df[tool]['output'].fillna(df[tool]['error']).astype(str).str[:500]
    if tool == "volesti":
        df['volesti']['volumes'] = df['volesti']['details'].apply(literal_eval).apply(lambda x: np.asarray([t['output'] for t in x]))
        a = df['volesti']['volumes'].apply(lambda x: x.mean())
        b = df['volesti']['volumes'].apply(lambda x: x.std())
        df_res[tool.capitalize()+"_meanstd"] = list(np.vstack((a, b)).T)
    df_res[tool.capitalize()+"_time"] = df[tool]['time']

df_res.head() 

,Function,Faza_output,Faza_time,Gubpi_output,Gubpi_time,Latte_output,Latte_time,Psi_output,Psi_time,Volesti_output,Volesti_meanstd,Volesti_time
0,(3.87*x**1+5.77*x**0) / (8.32*x**3 + 5.44*x**2...,"(0.6209815507002485, 0.7190312692318667)",95.899110,"(0.6775424712848596, 0.6788097626654503)",0.483498,Not a monomial: (8759501275235615/4 + (2341871...,0.061146,E[p_] = 100/99*Integrate[(387/100*xi1+577/100)...,0.023437,"(0.616943, 0.727933)","[0.6697733, 0.03167796447390521]","(0.10847139358520508, 0.13376188278198242)"
1,(2.67*x**1+3.94*x**0) / (2.08*x**1 + 2.78*x**0),"(1.3035252399983923, 1.4021111825192791)",2.971019,"(1.3829908706544114, 1.383145732964076)",0.432379,Not a monomial: (6260003482044989.0 + (4683743...,0.036545,E[p_] = 100/99*Boole[Integrate[1/(((-482469/31...,0.492784,"(1.26521, 1.39941)","[1.3258739999999998, 0.045871780257583226]","(0.1087343692779541, 0.13344430923461914)"
2,(0.15*x**0) / (5.08*x**2 + 5.3*x**1 + 9.11*x**0),"(0, 0.016205584149650985)",11.460795,"(0.011671671292629613, 0.011672594335843871)",0.439776,Not a monomial: (328222340842761728.0 + (18302...,0.031705,E[p_] = 1/Integrate[1/(-392563/2500+1/xi1*381/...,0.200462,"(0.0104545, 0.0121287)","[0.011488439999999999, 0.0005155557277346454]","(0.10523247718811035, 0.14924836158752441)"
3,(3.53*x**2+4.89*x**1+0.17*x**0) / (4.51*x**2 +...,"(0.47723272447183107, 0.5754865206866198)",16.182557,"(0.5411507819421031, 0.5422613034688918)",0.578539,Not a monomial: (20896702270999100.0 + (162489...,0.037204,E[p_] = 1/Integrate[(1/((((-389197/5000*xi1+21...,111.551891,"(0.469831, 0.617519)","[0.5467838, 0.03590879065855603]","(0.10309672355651855, 0.14017534255981445)"
4,(8.03*x**2+2.42*x**1+9.79*x**0) / (3.11*x**2 +...,"(1.0119278375331635, 1.1080609820988139)",17.220680,"(1.0653504907070543, 1.0658492424387842)",0.640732,Not a monomial: (2085729577425961/2 + (5150992...,0.042255,E[p_] = 1/Integrate[(1/((((-192874/625+-84579/...,140.077922,"(0.943715, 1.11774)","[1.0480953, 0.0498250734149986]","(0.11050629615783691, 0.14609694480895996)"


In [ ]:
df_res.to_csv(os.path.join(RESULT_DIR, f'result_{benchmark}_{int(time.time())}.csv'), index=False)